In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from my_functions import *
from LumFunc_miniJPAS import LumFunc_hist
from scipy.integrate import simpson

In [ ]:
w_central = central_wavelength()
nb_fwhm_Arr = nb_fwhm(range(60))

In [ ]:
split_converter = lambda s: np.array(s.split()).astype(float)
sum_flags = lambda s: np.sum(np.array(s.split()).astype(float))

cat = pd.read_csv('csv/minijpas.Flambda_photoz_gaia_7.csv', sep=',', header=1,
    converters={0: int, 1: int, 2: split_converter, 3: split_converter, 4: sum_flags,
    5: sum_flags})

cat = cat[np.array([len(x) for x in cat['FLUX_AUTO']]) != 0] # Drop bad rows due to bad query
cat = cat[(cat.FLAGS == 0) & (cat.MASK_FLAGS == 0)] # Drop flagged
cat = cat.reset_index()

pm_flx = np.stack(cat['FLUX_AUTO'].to_numpy()).T * 1e-19
pm_err = np.stack(cat['FLUX_RELERR_AUTO'].to_numpy()).T * pm_flx

In [ ]:
cat.dtypes

In [ ]:
mag = flux_to_mag(pm_flx, np.array(w_central.reshape(-1, 1)))

In [ ]:
plt.hist(mag[-2], bins=np.linspace(15, 26, 30))
plt.show()

In [ ]:
w_lya = 1215.67 # A

N_sources = len(cat['FLUX_AUTO'])
z_nb_Arr = w_central[:-4] / w_lya - 1
print(N_sources)

In [ ]:
pm_mask = mask_proper_motion(cat)
mag_mask = mag[-2] > 18

mask = pm_mask & mag_mask

In [ ]:
# Lya search
cont_est_lya, cont_err_lya = estimate_continuum(pm_flx, pm_err, IGM_T_correct=True)
line = is_there_line(pm_flx, pm_err, cont_est_lya, cont_err_lya, 30, mask=mask)
lya_lines, lya_cont_lines = identify_lines(line, pm_flx, pm_err, first=True)

# Other lines
cont_est_other, cont_err_other = estimate_continuum(pm_flx, pm_err, IGM_T_correct=False)
line_other = is_there_line(pm_flx, pm_err, cont_est_other, cont_err_other,
    160, obs=True, mask=mask)
other_lines = identify_lines(line_other, pm_flx, pm_err)

In [ ]:
z_min = 2.5
z_max = 3.5

z_Arr = np.zeros(N_sources)
z_Arr[np.where(np.array(lya_lines) != -1)] =\
    z_NB(np.array(lya_cont_lines)[np.where(np.array(lya_lines) != -1)])

##
lya_L = np.zeros(N_sources)
lya_R = np.zeros(N_sources)
lya_R2 = np.zeros(N_sources)
lya_L_err = np.zeros(N_sources) * 99
lya_R_err = np.zeros(N_sources) * 99

z_mask = (z_Arr > z_min) & (z_Arr < z_max)

nice_lya = nice_lya_select(
    lya_lines, other_lines, pm_flx, pm_err, cont_est_other, z_Arr, mask=z_mask
)

In [ ]:
len(np.where(nice_lya)[0])

In [ ]:
for k, src in enumerate(np.where(nice_lya)[0]):
    if k == 5: break
    fig = plt.figure(figsize=(8, 6))
    ax = plot_JPAS_source(pm_flx[:, src], pm_err[:, src])
    print(f'z_NB = {z_Arr[src]}')
    
    ax.plot(w_central[:56], cont_est_lya[:, src], ls='--', c='black')

    ax.axvline(w_central[lya_lines[src]], label='Selected NB')
    ax.legend(fontsize=13)
    plt.show()

In [ ]:
total_hist = np.zeros(15)
volume = 0

flambda_Arr = np.array([])
cont_Arr = np.array([])
z_sel_Arr = np.array([])

for nb_c in range(56):
    LAE_src = (np.array(lya_lines) == nb_c)
    cont = cont_est_lya[nb_c, nice_lya & LAE_src]
    flambda = pm_flx[nb_c, nice_lya & LAE_src] - cont
    z_sel = (w_central[nb_c] / w_lya - 1) * np.ones(len(flambda))

    flambda_Arr = np.concatenate((flambda_Arr, flambda))
    z_sel_Arr = np.concatenate((z_sel_Arr, z_sel))
    cont_Arr = np.concatenate((cont_Arr, cont))

    LF_bins, LF_hist, volume_i, bin_width = LumFunc_hist(
        flambda, w_central[nb_c], nb_fwhm_Arr[nb_c],
        n_bins=15, L_min=43, L_max=45.5
    )
    total_hist += LF_hist
    volume += volume_i
volume = z_volume(z_min, z_max, 0.895)

In [ ]:
EW_sel = flambda_Arr * 147 / cont_Arr / (1 + z_sel_Arr)

fig, ax = plt.subplots(figsize=(7, 6))

ax.hist(EW_sel, bins=np.linspace(0, 250, 15))

plt.show()

In [ ]:
correct = np.array([40.79057206, 22.44857194,  9.14136613,  5.49164951,  3.7504284 ,
        3.44561075,  3.37148001,  2.95610486,  2.93075819,  2.96819681,
        3.66416679,  3.19076351,  2.00235841,  1.49076097,  0.75414258])

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))

ax.errorbar(LF_bins, total_hist / volume / bin_width,
    yerr= total_hist ** 0.5 / volume / bin_width,
    marker='.', linestyle='', markersize=15, label='Uncorrected')
ax.errorbar(LF_bins, total_hist / volume / bin_width * correct,
    yerr= (total_hist * correct) ** 0.5 / volume / bin_width,
    marker='.', linestyle='', markersize=15, label='Corrected')

Lx = np.linspace(10 ** 42, 10 ** 46, 10000)
phistar1 = 3.33e-6
Lstar1 = 10 ** 44.65
alpha1 = -1.35
Phi1 = schechter(Lx, phistar1, Lstar1, alpha1) * Lx * np.log(10)

phistar2 = 10 ** -3.45
Lstar2 = 10 ** 42.93
alpha2 = -1.93
Phi2 = schechter(Lx, phistar2, Lstar2, alpha2) * Lx * np.log(10)

ax.plot(np.log10(Lx), Phi1, label='Spinoso 2020 (QSO)')
ax.plot(np.log10(Lx), Phi2, label='Sobral 2018 (SF)')

ax.set_yscale('log')
ax.set_xlabel(r'$L_{\mathrm{Ly}\alpha}$ (erg s$^{-1}$)', fontsize=15)
ax.set_ylabel(r'$\Phi$ (Mpc$^{-3}\,\Delta\logL^{-1}$)',
    fontsize=15)
ax.set_ylim(1e-8, 1e-3)
ax.set_xlim(42.5, 45.5)
ax.legend(fontsize=15)

plt.show()

In [ ]:
vol = z_volume(2, 3.5, 1)

L_integrate = np.linspace(10 ** 43.5, 10 ** 44.55, 1000)
Phi_integrate = schechter(L_integrate, phistar1, Lstar1, alpha1)

L_integrate2 = 10 ** np.linspace(43, 43.4, 1000)
Phi_integrate2 = schechter(L_integrate2, phistar1, Lstar1, alpha1)

Nt = simpson(Phi_integrate, L_integrate) * vol
Ne = Nt - simpson(Phi_integrate2, L_integrate2) * vol

print('N_expected = {0:0.0f} +- {1:0.0f}'.format(Nt, Ne))

In [ ]:
plt.hist(cat['zsp'][nice_lya])

In [ ]:
total_hist